In [1]:
import os, sys
import pandas as pd
import torch

In [2]:
!pip install deepface

     |████████████████████████████████| 61 kB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 12.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.2.0-py3-none-any.whl size=14273 sha256=a9eba14a9b4bf25141636eaf76185898fd4c22b12553af30e8457fd58977ca98
  Stored in directory: /root/.cache/pip/wheels/8c/17/ff/58721d1fabdb87c21a0529948cf39e2be9af90ddbe4ad65944
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


In [3]:
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 19 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=7f47d51e95628a52eb70204b8ee58f5473968e09479bab7ffc36b1ca3891c9b8
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
from drive.MyDrive.backend.all_models import predict_on_video, blink_on_video, detect_beard, detect_shades
from drive.MyDrive.backend.helper_functions import get_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2179072/45929032 bytes (4.7%)4620288/45929032 bytes (10.1%)7004160/45929032 bytes (15.2%)10297344/45929032 bytes (22.4%)13787136/45929032 bytes (30.0%)16932864/45929032 bytes (36.9%)20283392/45929032 bytes (44.2%)23658496/45929032 bytes (51.5%)26976256/45929032 bytes (58.7%)30130176/45929032 bytes (65.6%)33603584/45929032 bytes (73.2%)36864000/45929032 bytes (80.3%)

In [6]:
# load df from csv
test_df = pd.read_csv("drive/MyDrive/backend/deepfake_test_updated.csv", header=0)
test_df

,Name,Length (bytes),CreationTime,Real Labels,DFD (15 FPS),DFD (12 FPS),DFD (09 FPS),DFD (06 FPS),DFD (03 FPS),DFD (01 FPS),Blink (15 FPS) 0,Blink (15 FPS) 1,Blink (15 FPS) 2,Blink (15 FPS) 3,Beard: Age,Beard: Gender,Glasses
0,aassnaulhq.mp4,6987195,12/10/2019 2:21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aayfryxljh.mp4,10114027,12/10/2019 2:21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,acazlolrpz.mp4,4098352,12/10/2019 2:21,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adohdulfwb.mp4,3736581,12/10/2019 2:21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ahjnxtiamx.mp4,4259685,12/10/2019 2:21,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,uqvxjfpwdo.mp4,9083388,12/10/2019 2:24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,usqqvxcjmg.mp4,12498898,12/10/2019 2:24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,uubgqnvfdl.mp4,6677896,12/10/2019 2:24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,uvrzaczrbx.mp4,3480248,12/10/2019 2:24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Keep track of the current row for input
counter=0

In [8]:
# Change the directories as you see fit.

cwd = os.getcwd()
pre2 = cwd+"/drive/MyDrive/backend/data/test_videos"

# This is the directory of the 4 model outputs = 4 txt files.
# output_dir = "../src/pages/Home/VideoFilePlayer"

path_to_img="drive/MyDrive/backend/example_videos/temp/"
temp_img_shades=path_to_img+'p.png'
temp_img_beard=path_to_img+'o.png'

In [9]:
# Use GPU, if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
# Set up libraries
# sys.path.insert(0,cwd + "drive/MyDrive/backend/imports/blazeface")

# import libraries
from drive.MyDrive.backend.imports.blazeface.blazeface import BlazeFace

In [11]:
# Load blazeface training weights from directory
facedet = BlazeFace().to(device)
facedet.load_weights(cwd + "/drive/MyDrive/backend/imports/blazeface/blazeface.pth")
facedet.load_anchors(cwd + "/drive/MyDrive/backend/imports/blazeface/anchors.npy")
_ = facedet.train(False)

In [12]:
# Run all models

In [14]:
for i in range(0,100):
  video_path = pre2+'/'+str(test_df['Name'].iloc[i])

  DFD_score=predict_on_video(video_path, 15, device, facedet)
  test_df.loc[i, 'DFD (15 FPS)'] = DFD_score

  model_for_tests=get_model()
  blink_scores=blink_on_video(video_path, 15, facedet, model_for_tests)
  test_df.loc[i, 'Blink (15 FPS) 0'] = blink_scores[0]
  test_df.loc[i, 'Blink (15 FPS) 1'] = blink_scores[1]
  test_df.loc[i, 'Blink (15 FPS) 2'] = blink_scores[2]
  test_df.loc[i, 'Blink (15 FPS) 3'] = blink_scores[3]

  if(blink_scores[2]>0 or blink_scores[3]>0):
    beard_age, beard_gender=detect_beard(temp_img_beard)
    test_df.loc[i, 'Beard: Age'] = beard_age
    test_df.loc[i, 'Beard: Gender'] = beard_gender

    glasses_score=detect_shades(temp_img_shades)
    test_df.loc[i, 'Glasses'] = glasses_score

  else:
    test_df.loc[i, 'Beard: Age'] = -1
    test_df.loc[i, 'Beard: Gender'] = "N/A"
    test_df.loc[i, 'Glasses'] = -1

  test_df.to_csv("drive/MyDrive/backend/deepfake_test_updated.csv", index=False)

  print("\n---------"+str(i)+"---------\n")



predict_on_video()
0.5292167067527771

blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 48.66666666666667%
Amount of open eyes frames: 5.333333333333334%
Amount of closed eyes frames: 46.0%
[0.0, 48.67, 5.33, 46.0]

detect_beard()


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  3.90it/s]


   Age: 43
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
Top 5 Object Detection Predictions
bow_tie (9.77%)
sunglasses (9.12%)
mask (6.67%)
Band_Aid (6.51%)
saltshaker (5.46%)
There appears to be sunglasses. Larger glasses are hard to deepfake.

---------0---------


predict_on_video()
0.005198784172534943

blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 0.0%
Amount of open eyes frames: 88.66666666666667%
Amount of closed eyes frames: 11.333333333333332%
[0.0, 0.0, 88.67, 11.33]

detect_beard()


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.70it/s]


   Age: 47
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
Top 5 Object Detection Predictions
seat_belt (21.74%)
neck_brace (8.55%)
Band_Aid (7.81%)
sunscreen (7.37%)
bow_tie (5.45%)
No sunglasses detected.

---------1---------


predict_on_video()
0.733870267868042

blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 14.000000000000002%
Amount of open eyes frames: 9.333333333333334%
Amount of closed eyes frames: 76.66666666666667%
[0.0, 14.0, 9.33, 76.67]

detect_beard()


...

Action: gender: 100%|██████████| 2/2 [00:00<00:00,  3.94it/s]


   Age: 38
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
Top 5 Object Detection Predictions
bow_tie (9.83%)
Band_Aid (8.74%)
lipstick (6.93%)
shower_cap (6.62%)
bubble (6.17%)
No sunglasses detected.

---------98---------


predict_on_video()
0.9764018058776855

blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 0.0%
Amount of open eyes frames: 98.0%
Amount of closed eyes frames: 2.0%
[0.0, 0.0, 98.0, 2.0]

detect_beard()


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  3.88it/s]


   Age: 40
Gender: Woman


detect_shades()
Top 5 Object Detection Predictions
wig (32.94%)
sunscreen (21.94%)
mortarboard (5.37%)
shower_cap (4.27%)
Band_Aid (3.59%)
No sunglasses detected.

---------99---------



<Figure size 432x288 with 0 Axes>

In [16]:
for i in range(0,100):
  video_path = pre2+'/'+str(test_df['Name'].iloc[i])

  DFD_score_12=predict_on_video(video_path, 12, device, facedet)
  test_df.loc[i, 'DFD (12 FPS)'] = DFD_score_12

  DFD_score_09=predict_on_video(video_path, 9, device, facedet)
  test_df.loc[i, 'DFD (09 FPS)'] = DFD_score_09

  DFD_score_06=predict_on_video(video_path, 6, device, facedet)
  test_df.loc[i, 'DFD (06 FPS)'] = DFD_score_06

  DFD_score_03=predict_on_video(video_path, 3, device, facedet)
  test_df.loc[i, 'DFD (03 FPS)'] = DFD_score_03

  DFD_score_01=predict_on_video(video_path, 1, device, facedet)
  test_df.loc[i, 'DFD (01 FPS)'] = DFD_score_01

  test_df.to_csv("drive/MyDrive/backend/deepfake_test_updated.csv", index=False)

  print("\n---------"+str(i)+"---------\n")



predict_on_video()
0.536799967288971

predict_on_video()
0.536746084690094

predict_on_video()
0.550163745880127

predict_on_video()
0.5603461265563965

predict_on_video()
0.5487830638885498

---------0---------


predict_on_video()
0.005302454344928265

predict_on_video()
0.004884087946265936

predict_on_video()
0.004623390268534422

predict_on_video()
0.0031731855124235153

predict_on_video()
0.0177912637591362

---------1---------


predict_on_video()
0.7222848534584045

predict_on_video()
0.7273898124694824

predict_on_video()
0.7497237324714661

predict_on_video()
0.7564110159873962

predict_on_video()
0.7403458952903748

---------2---------


predict_on_video()
0.08641089498996735

predict_on_video()
0.09092440456151962

predict_on_video()
0.09917397797107697

predict_on_video()
0.12515884637832642

predict_on_video()
0.05480886623263359

---------3---------


predict_on_video()
0.9107993245124817

predict_on_video()
0.8855214715003967

predict_on_video()
0.9020385146141052

pre

In [ ]:
# eyeblink -> NOT a deepfake if 55<open<98
# beard -> NOT a deepfake if 20yr+ male
# shades -> NOT a deepfake if no shades detected
# eyeblink+beard
# eyeblink+shades
# beard+shades
# eyeblink+beard+shades